In [2]:
import os
import json
import gzip
import numpy as np
from tqdm import tqdm
from collections import defaultdict
import argparse

In [3]:
log_dir = "logs/PersONAL_v2/easy"
data_content_dir = "data/datasets/PersONAL/val/test_baselines/easy/content"

log_dir = "logs/PersONAL_v2/medium"
data_content_dir = "data/datasets/PersONAL/val/test_baselines/medium/content"

log_dir = "logs/PersONAL_v2/hard"
data_content_dir = "data/datasets/PersONAL/val/test_baselines/hard/content"

len(os.listdir(log_dir))

684

### Load Habitat Sim

In [5]:
import habitat_sim
from habitat_sim import Simulator

/mnt/anaconda3/envs/zson_39_22/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
scene_dataset_dir = "habitat-lab/data/scene_datasets"

scene = "svBbv1Pavdk"

img_size = (480, 640)
sensor_height = 0.88
hfov = 79.0         #TODO: Check if right
agent_height = 0.88
agent_radius = 0.18

In [7]:
sim_cfg = habitat_sim.SimulatorConfiguration()
sim_cfg.scene_dataset_config_file = os.path.join(scene_dataset_dir, "hm3d/hm3d_annotated_basis.scene_dataset_config.json")
sim_cfg.scene_id = scene

sim_cfg.enable_physics = False
sim_cfg.gpu_device_id = 0
sim_cfg.allow_sliding = False

sensor_specs = []
for name, sensor_type in zip(
    ["color", "depth", "semantic"],
    [
        habitat_sim.SensorType.COLOR,
        habitat_sim.SensorType.DEPTH,
        # habitat_sim.SensorType.SEMANTIC,
    ],
):
    sensor_spec = habitat_sim.CameraSensorSpec()
    sensor_spec.uuid = f"{name}_sensor"
    sensor_spec.sensor_type = sensor_type
    sensor_spec.resolution = [img_size[0], img_size[1]]
    sensor_spec.position = [0.0, sensor_height, 0.0]
    sensor_spec.hfov = hfov
    sensor_spec.sensor_subtype = habitat_sim.SensorSubType.PINHOLE
    sensor_specs.append(sensor_spec)

# create agent specifications
agent_cfg = habitat_sim.agent.AgentConfiguration(
    height=agent_height,
    radius=agent_radius,
    sensor_specifications=sensor_specs,
    action_space = {
        "move_forward": habitat_sim.agent.ActionSpec(
            "move_forward", habitat_sim.agent.ActuationSpec(amount=0.25)
        ),
        "turn_left": habitat_sim.agent.ActionSpec(
            "turn_left", habitat_sim.agent.ActuationSpec(amount=30.0)
        ),
        "turn_right": habitat_sim.agent.ActionSpec(
            "turn_right", habitat_sim.agent.ActuationSpec(amount=30.0)
        ),
    }
)

cfg = habitat_sim.Configuration(sim_cfg, [agent_cfg])

In [9]:
sim = habitat_sim.Simulator(cfg)

[12:43:00:826562]:[Metadata] AttributesManagerBase.h(352)::createFromJsonOrDefaultInternal : <Dataset>: Proposing JSON name : default.scene_dataset_config.json from original name : default| This file does not exist.
[12:43:00:826683]:[Metadata] AssetAttributesManager.cpp(121)::createObject : Asset attributes (capsule3DSolid:capsule3DSolid_hemiRings_4_cylRings_1_segments_12_halfLen_0.75_useTexCoords_false_useTangents_false) created and registered.
[12:43:00:826718]:[Metadata] AssetAttributesManager.cpp(121)::createObject : Asset attributes (capsule3DWireframe:capsule3DWireframe_hemiRings_8_cylRings_1_segments_16_halfLen_1) created and registered.
[12:43:00:826741]:[Metadata] AssetAttributesManager.cpp(121)::createObject : Asset attributes (coneSolid:coneSolid_segments_12_halfLen_1.25_rings_1_useTexCoords_false_useTangents_false_capEnd_true) created and registered.
[12:43:00:826760]:[Metadata] AssetAttributesManager.cpp(121)::createObject : Asset attributes (coneWireframe:coneWireframe_s

In [10]:
# # set the navmesh
# print(f"Pathfinder is Loaded: {sim.pathfinder.is_loaded}")
# navmesh_settings = habitat_sim.NavMeshSettings()
# navmesh_settings.set_defaults()
# navmesh_settings.agent_height = agent_height
# navmesh_settings.agent_radius = agent_radius
# navmesh_settings.agent_max_climb = 0.2
# navmesh_settings.cell_height = 0.2
# # navmesh_settings.include_static_objects = True
# navmesh_success = sim.recompute_navmesh(sim.pathfinder, navmesh_settings)
# print(f"Navmesh Recompute Success: {navmesh_success}")
# assert navmesh_success, "Failed to build the navmesh!"

### Geodesic DTG

##### Funcs

In [20]:
dist_btw_pts = lambda pt_1, pt_2: np.linalg.norm(pt_1 - pt_2)

def geo_dist(sim: Simulator, agent_pos: np.ndarray, obj_pos: np.ndarray):

    """
    Implementation follows habitat. 
    Refer to habitat-lab/habitat/sims/habitat_simulator.py -> HabitatSim.geodesic_distance
    """

    path = habitat_sim.MultiGoalShortestPath()
    if isinstance(obj_pos[0], np.ndarray):
        path.requested_ends = np.array(obj_pos, dtype=np.float32)
    else:
        path.requested_ends = np.array([np.array(obj_pos, dtype=np.float32)])
    
    path.requested_start = np.array(agent_pos, dtype=np.float32)
    
    sim.pathfinder.find_path(path)
    return path.geodesic_distance


def euc_dist(agent_pos, obj_pos):

    assert type(agent_pos) is np.ndarray
    assert type(obj_pos) is np.ndarray
    assert len(obj_pos.shape) == 2
    assert len(agent_pos.shape) == 1

    dists = np.apply_along_axis(lambda pt : dist_btw_pts(pt, agent_pos), axis=1, arr=obj_pos)
    return min(dists)


def get_geo_dtg(sim,
                final_pos, 
                obj_cat, obj_name,
                scene_name, scene_info,
                ):

    """
    Returns geodesic distance to object position and nearest viewpoint
    """

    obj_pos, obj_view_pts = [], []

    #Obtain the position of the target object and viewpoints
    goal_key = f"{scene_name}.basis.glb_{obj_cat}"
    for goal_instance in scene_info["goals_by_category"][goal_key]:

        if goal_instance["object_name"] != obj_name: continue   #Skip if not instance

        curr_view_pts = goal_instance["view_points"]
        curr_view_pts = [pt["agent_state"]["position"] for pt in curr_view_pts]

        obj_pos.append(goal_instance["position"])
        obj_view_pts.append(curr_view_pts)

    obj_pos, obj_view_pts = np.array(obj_pos), np.vstack(obj_view_pts)

    #Calculate distance to goal
    dtg_obj = geo_dist(sim, final_pos, obj_pos)
    dtg_vw_pt = geo_dist(sim, final_pos, obj_view_pts)
    
    return dtg_obj, dtg_vw_pt

In [22]:
get_geo_dtg(sim, final_pos,
                    obj_cat = ep_info["object_category"],
                    obj_name = ep_info["object_id"],
                    scene_name = scene_id,
                    scene_info = scene_info)

(1.2406935691833496, 0.009194165468215942)

##### Class

In [13]:
import habitat_sim
import numpy as np
from tqdm import tqdm

In [ ]:
class Traj_Metrics:

    def __init__(self,
                 scene_name: str,
                 data_content_dir: str,
                 success_thresh = 0.1,      #Distance to nearest viewpoint
                 scene_dataset_cfg = "habitat-lab/data/scene_datasets/hm3d/hm3d_annotated_basis.scene_dataset_config.json",
                 agent_height = 0.88, agent_radius = 0.18,
                 recomp_navmesh = False
                 ):
        
        self.scene_name = scene_name
        self.success_thresh = success_thresh
        
        #Load scene episodes
        scene_info_path = os.path.join(data_content_dir, f"{scene_name}.json.gz")
        with gzip.open(scene_info_path, "r") as f:
            self.scene_info = json.load(f)
        
        #Initialize Simulator
        self._init_sim(scene_dataset_cfg, 
                       agent_height, agent_radius,
                       recomp_navmesh)

    # ---
    # Habitat Simulator
    # ---

    def _init_sim(self,
                  scene_dataset_cfg,
                  agent_height, agent_radius,
                  recomp_navmesh):

        sim_cfg = habitat_sim.SimulatorConfiguration()
        sim_cfg.scene_dataset_config_file = scene_dataset_cfg
        sim_cfg.scene_id = self.scene_name

        sim_cfg.enable_physics = False
        sim_cfg.gpu_device_id = 0
        sim_cfg.allow_sliding = False

        sensor_specs = []
        # for name, sensor_type in zip(
        #     ["color", "depth", "semantic"],
        #     [
        #         habitat_sim.SensorType.COLOR,
        #         habitat_sim.SensorType.DEPTH,
        #         # habitat_sim.SensorType.SEMANTIC,
        #     ],
        # ):
        #     sensor_spec = habitat_sim.CameraSensorSpec()
        #     sensor_spec.uuid = f"{name}_sensor"
        #     sensor_spec.sensor_type = sensor_type
        #     sensor_spec.resolution = [img_size[0], img_size[1]]
        #     sensor_spec.position = [0.0, sensor_height, 0.0]
        #     sensor_spec.hfov = hfov
        #     sensor_spec.sensor_subtype = habitat_sim.SensorSubType.PINHOLE
        #     sensor_specs.append(sensor_spec)

        # create agent specifications
        agent_cfg = habitat_sim.agent.AgentConfiguration(
            height=agent_height,
            radius=agent_radius,
            sensor_specifications=sensor_specs,
            action_space = {
                "move_forward": habitat_sim.agent.ActionSpec(
                    "move_forward", habitat_sim.agent.ActuationSpec(amount=0.25)
                ),
                "turn_left": habitat_sim.agent.ActionSpec(
                    "turn_left", habitat_sim.agent.ActuationSpec(amount=30.0)
                ),
                "turn_right": habitat_sim.agent.ActionSpec(
                    "turn_right", habitat_sim.agent.ActuationSpec(amount=30.0)
                ),
            }
        )

        cfg = habitat_sim.Configuration(sim_cfg, [agent_cfg])
        self.sim = habitat_sim.Simulator(cfg)

        if recomp_navmesh:
            # set the navmesh
            print(f"Pathfinder is Loaded: {self.sim.pathfinder.is_loaded}")
            navmesh_settings = habitat_sim.NavMeshSettings()
            navmesh_settings.set_defaults()
            navmesh_settings.agent_height = agent_height
            navmesh_settings.agent_radius = agent_radius
            navmesh_success = self.sim.recompute_navmesh(self.sim.pathfinder, navmesh_settings)
            print(f"Navmesh Recompute Success: {navmesh_success}")
            assert navmesh_success, "Failed to build the navmesh!"

    def _close_sim(self):
        self.sim.close()

    # ----
    # Distance Calculation
    # ----

    def _geo_dist(self, agent_pos: np.ndarray, obj_pos: np.ndarray):

        """
        Implementation follows habitat. 
        Refer to habitat-lab/habitat/sims/habitat_simulator.py -> HabitatSim.geodesic_distance
        """

        path = habitat_sim.MultiGoalShortestPath()
        if isinstance(obj_pos[0], np.ndarray):
            path.requested_ends = np.array(obj_pos, dtype=np.float32)
        else:
            path.requested_ends = np.array([np.array(obj_pos, dtype=np.float32)])
        
        path.requested_start = np.array(agent_pos, dtype=np.float32)
        
        self.sim.pathfinder.find_path(path)
        return path.geodesic_distance

    def _euc_dist(self, agent_pos: np.ndarray, obj_pos: np.ndarray):

        assert len(obj_pos.shape) == 2
        assert len(agent_pos.shape) == 1

        dists = np.apply_along_axis(lambda pt : dist_btw_pts(pt, agent_pos), axis=1, arr=obj_pos)
        return min(dists)
    
    def _dist_btw_pts(self, pt_1: np.ndarray, pt_2: np.ndarray):
        return np.linalg.norm(pt_1 - pt_2)
        

    # ---
    # Metric : Distance to Goal (Geodesic and Euclidean)
    # ---

    def get_geo_dtg(self,
                    final_pos, 
                    obj_cat, obj_name,
                    ):

        """
        Returns geodesic distance to object position and nearest viewpoint
        """

        obj_pos, obj_view_pts = [], []

        #Obtain the position of the target object and viewpoints
        goal_key = f"{self.scene_name}.basis.glb_{obj_cat}"
        for goal_instance in self.scene_info["goals_by_category"][goal_key]:

            if goal_instance["object_name"] != obj_name: continue   #Skip if not instance

            curr_view_pts = goal_instance["view_points"]
            curr_view_pts = [pt["agent_state"]["position"] for pt in curr_view_pts]

            obj_pos.append(goal_instance["position"])
            obj_view_pts.append(curr_view_pts)

        obj_pos, obj_view_pts = np.array(obj_pos), np.vstack(obj_view_pts)

        #Calculate distance to goal
        dtg_obj = geo_dist(final_pos, obj_pos)
        dtg_vw_pt = geo_dist(final_pos, obj_view_pts)
        
        return dtg_obj, dtg_vw_pt
    
    def get_euc_dtg(self, 
                    final_pos, 
                    obj_cat, obj_name):

        obj_pos, obj_view_pts = [], []

        #Obtain the position of the target object and viewpoints
        goal_key = f"{self.scene_name}.basis.glb_{obj_cat}"
        for goal_instance in self.scene_info["goals_by_category"][goal_key]:

            if goal_instance["object_name"] != obj_name: continue   #Skip if not instance

            curr_view_pts = goal_instance["view_points"]
            curr_view_pts = [pt["agent_state"]["position"] for pt in curr_view_pts]

            obj_pos.append(goal_instance["position"])
            obj_view_pts.append(curr_view_pts)

        obj_pos, obj_view_pts = np.array(obj_pos), np.vstack(obj_view_pts)
        # return obj_pos, obj_view_pts

        obj_pos, obj_view_pts = obj_pos[:, [0, 2]], obj_view_pts[:, [0, 2]]

        #Calculate distance to goal
        view_pt_dists = np.apply_along_axis(lambda v: dist_btw_pts(v, final_pos), axis=1, arr=obj_view_pts)
        dtg_vw_pt = min(view_pt_dists)

        obj_dists = np.apply_along_axis(lambda v: dist_btw_pts(v, final_pos), axis=1, arr=obj_pos)
        dtg_obj = min(obj_dists)

        return dtg_obj, dtg_vw_pt

    # ---
    # Calculate Metrics : SR, SPL, DTG
    # ---

    def eval_metrics(self, traj_path: str, episode_id: str):

        #Read trajectory
        with open(traj_path, "r") as f:
            traj = f.readlines()

        traj = [[float(s_sub) for s_sub in s.strip().split(",")] for s in traj]
        traj = np.array(traj)

        #Final Position of agent
        final_pos = traj[-1][1:]

        #Geodesic Distance covered by trajectory
        traj_path_dist = sum(
            self.geo_dist(pt_1, pt_2)
            for pt_1, pt_2 in zip(traj[:-1], traj[1:])
        )

        #GT : Obtain episode
        found_ep = False
        for ep_info in self.scene_info["episodes"]:
            if ep_info["episode_id"] == episode_id:
                found_ep = True
                break

        assert found_ep, f"Episode with id {episode_id} not found in scene {self.scene_name}"

        #Get distance to object and nearest viewpoint
        dtg_obj, dtg_vw = self._get_geo_dtg(final_pos = final_pos,
                                            obj_cat = ep_info["object_category"],
                                            obj_name = ep_info["object_id"])
        
        #Calculate metrics
        success, spl = 0, 0
        if dtg_vw <= self.success_thresh:
            success = 1

            #Get SPL
            gt_path_dist = ep_info["info"]["geodesic_distance"]
            spl = gt_path_dist / max(gt_path_dist, traj_path_dist)


        return success, spl, dtg_obj, dtg_vw

In [7]:
logs_by_scene = {}

#Arrange logs by the scene name. This is done to run the sim per scene.
for f_name in tqdm(os.listdir(log_dir)):

    if (not f_name.endswith(".txt")) or (not f_name.__contains__("_")): continue

    scene_name = f_name.split("_")[-1].replace(".txt", "")
    if scene_name in logs_by_scene:
        logs_by_scene[scene_name].append( os.path.join(log_dir, f_name) )
    else:
        logs_by_scene[scene_name] = [ os.path.join(log_dir, f_name) ]

100%|██████████| 684/684 [00:00<00:00, 299812.30it/s]


In [ ]:
success_vals = []
spl_vals = []
dtg_obj_vals = []
dtg_vw_vals = []

num_eps = 0

for scene in logs_by_scene:

    print(f"Scene : {scene}")

    evaluator = Traj_Metrics(scene_name = scene,
                                data_content_dir = data_content_dir,
                                success_thresh = 0.1)
    
 
    for f_log in tqdm(logs_by_scene[scene]):

        episode_id = f_log.split("/")[-1].split("_")[0]

        success, spl, dtg_obj, dtg_vw = evaluator.eval_metrics(traj_path = f_log,
                                                               episode_id = episode_id)
        
        success_vals.append(success)
        spl_vals.append(spl)
        dtg_obj_vals.append(dtg_obj)
        dtg_vw_vals.append(dtg_vw)

        num_eps += 1

    evaluator._close_sim()
    

print(f"Success Rate : {sum(success_vals)}/{num_eps} -> {(sum(success)/num_eps) * 100} %")
print(f"SPL : {sum(spl_vals)/num_eps}")
print(f"DTG (obj) : {sum(dtg_obj_vals)/num_eps}")
print(f"DTG (vw_pt) : {sum(dtg_vw_vals)/num_eps}")

In [9]:
scene_name

'5cdEh9F2hJL'

In [ ]:
evaluator = Traj_Metrics(scene_name = scene_name,
                            data_content_dir = data_content_dir,
                            success_thresh = 0.1)

In [36]:
scene

'a8BtkwhxdRV'

In [37]:
logs_by_scene[scene]

['logs/PersONAL_v2/hard/3892_a8BtkwhxdRV.txt',
 'logs/PersONAL_v2/hard/3872_a8BtkwhxdRV.txt',
 'logs/PersONAL_v2/hard/3954_a8BtkwhxdRV.txt',
 'logs/PersONAL_v2/hard/3963_a8BtkwhxdRV.txt',
 'logs/PersONAL_v2/hard/3921_a8BtkwhxdRV.txt',
 'logs/PersONAL_v2/hard/3976_a8BtkwhxdRV.txt',
 'logs/PersONAL_v2/hard/3984_a8BtkwhxdRV.txt',
 'logs/PersONAL_v2/hard/3953_a8BtkwhxdRV.txt',
 'logs/PersONAL_v2/hard/3946_a8BtkwhxdRV.txt',
 'logs/PersONAL_v2/hard/3897_a8BtkwhxdRV.txt',
 'logs/PersONAL_v2/hard/3916_a8BtkwhxdRV.txt',
 'logs/PersONAL_v2/hard/3871_a8BtkwhxdRV.txt',
 'logs/PersONAL_v2/hard/3869_a8BtkwhxdRV.txt',
 'logs/PersONAL_v2/hard/3860_a8BtkwhxdRV.txt',
 'logs/PersONAL_v2/hard/3923_a8BtkwhxdRV.txt',
 'logs/PersONAL_v2/hard/3962_a8BtkwhxdRV.txt',
 'logs/PersONAL_v2/hard/3904_a8BtkwhxdRV.txt',
 'logs/PersONAL_v2/hard/3870_a8BtkwhxdRV.txt',
 'logs/PersONAL_v2/hard/3945_a8BtkwhxdRV.txt']

### Rough

In [136]:
scene_id, episode_id = "q3zU7Yy5E5s", "84"

scene_info_path = os.path.join(data_content_dir, f"{scene_id}.json.gz")
print(f"Scene : {scene_id}, Episode : {episode_id}")

with gzip.open(scene_info_path, "r") as f_scene:
    scene_info = json.load(f_scene)

ids = []
for ep in scene_info["episodes"]:

    print(ep["object_id"], ep["object_category"])

    ids.append(ep["episode_id"])

    if ep["episode_id"] == episode_id:
        print(f"Found!")
        break

    # if ep["object_id"] == "blanket_207":
    #     print(f"Found thru obj ID!")
    #     break

Scene : q3zU7Yy5E5s, Episode : 84
armchair_118 armchair
picture_109 picture
armchair_119 armchair
pillow_90 pillow
rug_96 rug
heater_17 heater
window glass_6 window glass
bath sink_68 bath sink
glass_100 glass
sink_197 sink
oven and stove_217 oven and stove
oven and stove_217 oven and stove
sink_197 sink
oven and stove_217 oven and stove
glass_100 glass
mirror_309 mirror
mirror_309 mirror
pillow_92 pillow
kitchen cabinet_223 kitchen cabinet
kitchen cabinet_223 kitchen cabinet
kitchen cabinet_223 kitchen cabinet
pillow_92 pillow
armchair_428 armchair
mirror_418 mirror
picture_464 picture
heater_435 heater
bed sheet_447 bed sheet
mirror_418 mirror
heater_435 heater


In [114]:
ids

['58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '66',
 '67',
 '68',
 '69',
 '75',
 '78',
 '79']

In [94]:
for goal in scene_info["goals_by_category"]:

    goal_infos = scene_info["goals_by_category"][goal]

    for goal_info in goal_infos:
        print(f"{goal_info['object_name']}, {goal_info['object_category']}, {goal_info['object_name']}")
    # break

bathrobe_27, bathrobe, bathrobe_27
bed_142, bed, bed_142
bed_1178, bed, bed_1178
blanket_272, blanket, blanket_272
tv_113, tv, tv_113
tv_1182, tv, tv_1182
toilet_20, toilet, toilet_20
window shutter_259, window shutter, window shutter_259
boxes_1317, box, boxes_1317
board_1377, board, board_1377
board_1467, board, board_1467
container_1389, container, container_1389
chair_1450, chair, chair_1450


In [92]:
goal_info

[{'position': [-0.58496, 4.41101, -3.23173],
  'radius': None,
  'object_id': 27,
  'object_name': 'bathrobe_27',
  'object_name_id': None,
  'object_category': 'bathrobe',
  'floor_id': '2',
  'room_id': None,
  'room_name': None,
  'view_points': [{'agent_state': {'position': [-0.2087, 3.36917, -2.95582],
     'rotation': [0.0, 0.45202, -0.0, 0.89201]},
    'iou': 0.5719},
   {'agent_state': {'position': [0.0413, 3.35388, -3.20582],
     'rotation': [0.0, 0.69234, -0.0, 0.72158]},
    'iou': 0.42799},
   {'agent_state': {'position': [0.0413, 3.35388, -2.95582],
     'rotation': [0.0, 0.54627, -0.0, 0.83761]},
    'iou': 0.3825},
   {'agent_state': {'position': [-0.4587, 3.38754, -2.70582],
     'rotation': [0.0, 0.11754, -0.0, 0.99307]},
    'iou': 0.36852},
   {'agent_state': {'position': [-0.2087, 3.37698, -2.70582],
     'rotation': [0.0, 0.30554, -0.0, 0.95218]},
    'iou': 0.3547},
   {'agent_state': {'position': [0.0413, 3.35745, -2.70582],
     'rotation': [0.0, 0.42244, -0.0,

In [90]:
scene_info["goals_by_category"][goal]

[{'position': [-0.58496, 4.41101, -3.23173],
  'radius': None,
  'object_id': 27,
  'object_name': 'bathrobe_27',
  'object_name_id': None,
  'object_category': 'bathrobe',
  'floor_id': '2',
  'room_id': None,
  'room_name': None,
  'view_points': [{'agent_state': {'position': [-0.2087, 3.36917, -2.95582],
     'rotation': [0.0, 0.45202, -0.0, 0.89201]},
    'iou': 0.5719},
   {'agent_state': {'position': [0.0413, 3.35388, -3.20582],
     'rotation': [0.0, 0.69234, -0.0, 0.72158]},
    'iou': 0.42799},
   {'agent_state': {'position': [0.0413, 3.35388, -2.95582],
     'rotation': [0.0, 0.54627, -0.0, 0.83761]},
    'iou': 0.3825},
   {'agent_state': {'position': [-0.4587, 3.38754, -2.70582],
     'rotation': [0.0, 0.11754, -0.0, 0.99307]},
    'iou': 0.36852},
   {'agent_state': {'position': [-0.2087, 3.37698, -2.70582],
     'rotation': [0.0, 0.30554, -0.0, 0.95218]},
    'iou': 0.3547},
   {'agent_state': {'position': [0.0413, 3.35745, -2.70582],
     'rotation': [0.0, 0.42244, -0.0,

### Metrics Calculation

In [13]:
dist_btw_pts = lambda pt_1, pt_2: np.linalg.norm(pt_1 - pt_2)

def get_euc_dtg(final_pos, obj_cat, obj_name,
            scene_name, scene_info,
            to_vw_pt = True):

    obj_pos, obj_view_pts = [], []

    #Obtain the position of the target object and viewpoints
    goal_key = f"{scene_name}.basis.glb_{obj_cat}"
    for goal_instance in scene_info["goals_by_category"][goal_key]:

        if goal_instance["object_name"] != obj_name: continue   #Skip if not instance

        curr_view_pts = goal_instance["view_points"]
        curr_view_pts = [pt["agent_state"]["position"] for pt in curr_view_pts]

        obj_pos.append(goal_instance["position"])
        obj_view_pts.append(curr_view_pts)

    obj_pos, obj_view_pts = np.array(obj_pos), np.vstack(obj_view_pts)
    # return obj_pos, obj_view_pts

    obj_pos, obj_view_pts = obj_pos[:, [0, 2]], obj_view_pts[:, [0, 2]]

    #Calculate distance to goal
    if to_vw_pt:
        view_pt_dists = np.apply_along_axis(lambda v: dist_btw_pts(v, final_pos), axis=1, arr=obj_view_pts)
        dtg = min(view_pt_dists)

    else:
        obj_dists = np.apply_along_axis(lambda v: dist_btw_pts(v, final_pos), axis=1, arr=obj_pos)
        dtg = min(obj_dists)

    
    
    return dtg
    

In [14]:
dist_thresh = 0.1
dtg_to_vw = True

In [ ]:
success = 0
spl = 0

keys = []
spl_vals = []
dtg_vals = []

num_eps = 0

for f_name in tqdm(os.listdir(log_dir)):

    if (not f_name.endswith(".txt")) or (not f_name.__contains__("_")): continue

    #Trajetory
    f_path = os.path.join(log_dir, f_name)
    with open(f_path, "r") as f:
        traj = f.readlines()

    traj = [[float(s_sub) for s_sub in s.strip().split(",")] for s in traj]
    traj = np.array(traj)
    final_pos = traj[-1][1:]
    traj = traj[:, [1, 3]]

    deltas = traj[1:, :] - traj[:-1, :]
    traj_dist = np.linalg.norm(deltas, axis=1).sum()

    #Ground Truth
    scene_id, episode_id = f_name.split("_")[-1][:-4], f_name.split("_")[0]
    scene_info_path = os.path.join(data_content_dir, f"{scene_id}.json.gz")

    with gzip.open(scene_info_path, "r") as f_scene:
        scene_info = json.load(f_scene)

    #Obtain episode info
    found_ep = False
    for ep_info in scene_info["episodes"]:
        if ep_info["episode_id"] == episode_id:
            found_ep = True
            break

    assert found_ep

    #Distance to Goal (nearest viewpoint)
    dtg = get_euc_dtg(final_pos = traj[-1],
                    obj_cat = ep_info["object_category"],
                    obj_name = ep_info["object_id"],
                    scene_name = scene_id,
                    scene_info = scene_info,
                    to_vw_pt = dtg_to_vw)

    dtg_vals.append(dtg)
    if dtg <= dist_thresh:

        success += 1
    
        #Success Path Length (SPL)
        geo_dist = ep_info["info"]["geodesic_distance"]
        spl_ratio = geo_dist / max(geo_dist, traj_dist)
        spl_vals.append(spl_ratio)

        if num_eps > 3:
            break

    num_eps += 1
    # break

    


print(f"Success Rate : {success}/{num_eps} -> {(success/num_eps) * 100} %")
print(f"SPL : {sum(spl_vals)/num_eps}")
print(f"DTG : {sum(dtg_vals)/num_eps}")

  2%|▏         | 13/684 [00:00<00:13, 50.29it/s]

[12:43:55:405434]:[Sim] Simulator.cpp(65)::~Simulator : Deconstructing Simulator
[12:43:55:405515]:[Physics] PhysicsManager.cpp(50)::~PhysicsManager : Deconstructing PhysicsManager
[12:43:55:405603]:[Scene] SceneManager.h(25)::~SceneManager : Deconstructing SceneManager
[12:43:55:405615]:[Scene] SceneGraph.h(25)::~SceneGraph : Deconstructing SceneGraph
[12:43:55:405850]:[Scene] SemanticScene.h(47)::~SemanticScene : Deconstructing SemanticScene
[12:43:55:410191]:[Gfx] Renderer.cpp(72)::~Impl : Deconstructing Renderer
[12:43:55:410230]:[Gfx] WindowlessContext.h(17)::~WindowlessContext : Deconstructing WindowlessContext


  4%|▍         | 27/684 [00:00<00:16, 39.16it/s]

Success Rate : 2/27 -> 7.4074074074074066 %
SPL : 0.0384169360206596
DTG : 5.29740243080281


In [90]:
obj_pos, obj_vw_pts = get_dtg(traj[-1],
                                ep_info["object_category"],
                                ep_info["object_id"],
                                scene_id,
                                scene_info,
                                dtg_to_vw)

In [104]:
obj_pos

array([[-0.18356,  0.62392,  1.1219 ]])

In [105]:
final_pos

array([-0.50122237,  0.19491982, -0.04245704])

In [106]:
dtg

0.011389836920105041

In [21]:
obj_vw_pts

array([[ 1.49191,  0.0184 , -0.15212],
       [ 1.74191,  0.0184 , -0.15212],
       [ 1.24191,  0.0184 , -0.15212],
       [ 1.99191,  0.0184 , -0.15212],
       [ 0.99191,  0.0184 , -0.15212],
       [ 1.49191,  0.0184 ,  0.09788],
       [ 1.74191,  0.0184 ,  0.09788],
       [ 1.24191,  0.0184 ,  0.09788],
       [ 2.24191,  0.0184 , -0.15212],
       [ 1.99191,  0.0184 ,  0.09788],
       [ 0.74191,  0.0184 , -0.15212],
       [ 0.99191,  0.0184 ,  0.09788],
       [ 2.49191,  0.0184 , -0.15212],
       [ 2.24191,  0.0184 ,  0.09788],
       [ 1.49191,  0.0184 ,  0.34788],
       [ 1.74191,  0.0184 ,  0.34788],
       [ 1.24191,  0.0184 ,  0.34788],
       [ 0.74191,  0.0184 ,  0.09788],
       [ 0.49191,  0.0184 , -0.15212],
       [ 0.49191,  0.0184 , -0.40212],
       [ 1.99191,  0.0184 ,  0.34788],
       [ 0.99191,  0.0184 ,  0.34788],
       [ 2.49191,  0.0184 ,  0.09788],
       [ 2.74191,  0.0184 , -0.15212],
       [ 2.24191,  0.0184 ,  0.34788],
       [ 0.49191,  0.0184